In [1]:
import numpy as np

# Baum-Welch algorithm implementation
def BaumWelch(N, A, B, Pi, T):
    """
    :param N: Number of model states
    :param A: Initial transition probability matrix
    :param B: Initial output probability matrix
    :param Pi: Initial state probability matrix
    :param T: Training data
    :return: Re-estimated A, B, and Pi matrices
    """

    # Initialization step
    alpha = np.zeros((T.shape[0], N))
    beta = np.zeros((T.shape[0], N))
    gamma = np.zeros((T.shape[0], N))
    xi = np.zeros((T.shape[0] - 1, N, N))
    iterations = 0
    oldLogProb = -np.inf
    newLogProb = 0

    while iterations < 1000 and newLogProb > oldLogProb:
        oldLogProb = newLogProb

        # E-step
        for t in range(T.shape[0]):
            if t == 0:
                alpha[t, :] = Pi * B[:, T[t]]
            else:
                alpha[t, :] = np.sum(alpha[t - 1, :].reshape((1, -1)) * A, axis=1) * B[:, T[t]]

            if t == T.shape[0] - 1:
                beta[t, :] = 1
            else:
                beta[t, :] = np.sum(A * B[:, T[t + 1]] * beta[t + 1, :], axis=1)

            gamma[t, :] = alpha[t, :] * beta[t, :]
            gamma[t, :] /= (np.sum(gamma[t, :]) + 1e-9)

            if t != T.shape[0] - 1:
                xi[t, :, :] = alpha[t, :].reshape((-1, 1)) * A * B[:, T[t + 1]] * beta[t + 1, :]
                xi[t, :, :] /= (np.sum(xi[t, :, :]) + 1e-9)

        # M-step
        Pi = gamma[0, :]
        A = np.sum(xi, axis=0) / (np.sum(gamma[:-1, :], axis=0) + 1e-9).reshape((-1, 1))
        B = np.zeros((N, 2))
        for i in range(N):
            for j in range(2):
                B[i, j] = np.sum(gamma[T == j, i]) / (np.sum(gamma[:, i]) + 1e-9)

        newLogProb = np.sum(np.log(np.sum(alpha[T.shape[0] - 1, :])))
        iterations += 1

    return A, B, Pi

# Training data
T = np.array([0, 1, 0, 1, 0, 1])  # Example training data

# Initial model parameters
N = 2
A = np.array([[0.5, 0.5], [0.5, 0.5]])  # Initial transition probability matrix
B = np.array([[0.5, 0.5], [0.5, 0.5]])  # Initial output probability matrix
Pi = np.array([0.5, 0.5]) # Initial state probability matrix

#Train model using Baum-Welch algorithm
A, B, Pi = BaumWelch(N, A, B, Pi, T)

#Testing data
test_T = np.array([0, 1, 0, 1]) # Example testing data

#Run Vertebi to get likely hidden state sequence
def viterbi(T, N, A, B, Pi):
    delta = np.zeros((T.shape[0], N))
    psi = np.zeros((T.shape[0], N))
    # Initialization step
    delta[0, :] = np.log(Pi) + np.log(B[:, T[0]])
    psi[0, :] = 0

    # Recursion step
    for t in range(1, T.shape[0]):
        delta[t, :] = np.max(delta[t - 1, :].reshape((1, -1)) + np.log(A), axis=1) + np.log(B[:, T[t]])
        psi[t, :] = np.argmax(delta[t - 1, :].reshape((1, -1)) + np.log(A), axis=1)

    # Termination step
    q = np.zeros(T.shape[0], dtype=int)
    q[-1] = np.argmax(delta[-1, :])
    logProb = np.max(delta[-1, :])
    
    for t in range(T.shape[0] - 2, -1, -1):
        q[t] = psi[t + 1, int(q[t + 1])]
    return q, logProb
    
#Test newly trained model using Viterbi algorithm on testing data
q, logProb = viterbi(test_T, N, A, B, Pi)

print("Most likely hidden state sequence for testing data:")
print(q)
print("Log probability of the sequence:")
print(logProb)

Most likely hidden state sequence for testing data:
[0 0 0 0]
Log probability of the sequence:
-inf


C:\Users\jenny\AppData\Local\Temp\ipykernel_19124\4006001471.py:78: RuntimeWarning: divide by zero encountered in log
  delta[0, :] = np.log(Pi) + np.log(B[:, T[0]])
C:\Users\jenny\AppData\Local\Temp\ipykernel_19124\4006001471.py:83: RuntimeWarning: divide by zero encountered in log
  delta[t, :] = np.max(delta[t - 1, :].reshape((1, -1)) + np.log(A), axis=1) + np.log(B[:, T[t]])
C:\Users\jenny\AppData\Local\Temp\ipykernel_19124\4006001471.py:84: RuntimeWarning: divide by zero encountered in log
  psi[t, :] = np.argmax(delta[t - 1, :].reshape((1, -1)) + np.log(A), axis=1)
